In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import time

In [14]:
NB_Cars_MAX = 100
EXPORT_PATH = "./exports/"

In [15]:
url = "https://www.cars.com/shopping/results/?list_price_max=&makes[]=jaguar&maximum_distance=20&page=1&page_size=100&sort=list_price_desc&stock_type=new&zip="

options = webdriver.ChromeOptions()
#options.add_argument('headless')

options.add_argument('--disable-blink-features=AutomationControlled')
browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver",options=options)

browser.get(url)

time.sleep(2) #wait till all the content will be charged

##-------------##

#get the nb of results 

soup = BeautifulSoup(browser.page_source, 'html.parser') #first param: page html; 2 param: type of parser

nb_elements =soup.find("span",class_ = "total-filter-count").get_text(strip = True)#strip = true: remove all other characters
nb_elements = int((re.search('^(.*?)matches',nb_elements).group(1)).replace(" ","").replace(",",""))
nb_elements


soup = BeautifulSoup(browser.page_source, 'html.parser') 

browser.quit() #quit the navigator





/tmp/ipykernel_20812/1798284672.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver",options=options)


In [16]:
# Get items
items = soup.find_all("div",{"class":"vehicle-card"})
print(len(items))


101


In [17]:
def clean_text(str):
    return str.replace("\n","").replace("\t","")

In [18]:
import re
item = items[10]
reference = item.find("h2",{"class":"title"}).get_text(strip = True)
reference
car_dealer_name = clean_text(item.find("div",{"class":"dealer-name"}).get_text(strip = True))
#car_dealer_name

price = item.find("span",{"class":"primary-price"}).get_text(strip = True)
#price  
#pay_per_month = item.find("span",{"class":"js-estimated-monthly-payment-formatted-value-with-abr"}).get_text()
#item.xpath("//a[@class='sds-link']/span").text()
#item.find("span",{"class":"js-estimated-monthly-payment-formatted-value-with-abr"}).text()
#pay_per_month

#details = item.find("div",{"class":"list-item-tooltip-container"}).findChildren()
pay_per_month = clean_text((item.find("a",{"class":"sds-link"}).get_text()))


img_url=item.find('div',{"data-index":"0"}).find('img').attrs['data-src']
img_url

'https://platform.cstatic-images.com/large/in/v2/db401d26-17c7-5c54-9c13-b9e40e8a08eb/e0fd855f-f5d2-49c2-994e-47a5568ee76e/-6ZwC7rQn3dUvs0IzWzBP9eBT_w.jpg'

In [19]:
# store items scraped into a dict inside a list
cars = []
for item in items:
    reference = item.find("h2",{"class":"title"}).get_text(strip = True)
    car_dealer_name = clean_text(item.find("div",{"class":"dealer-name"}).get_text(strip = True))

    price = item.find("span",{"class":"primary-price"}).get_text()
    img_url=item.find('div',{"data-index":"0"}).find('img',{"class":"vehicle-image"}).get('data-src')
    #pay_per_month = clean_text((item.find("a",{"class":"sds-link"}).get_text(strip = True)))
    car = {
        "reference":reference,
        "car_dealer_name":car_dealer_name,
        "price":price,
        "img_url":img_url
        #"rating":rating,
        #"pay_per_month":pay_per_month

    }
    cars.append(car)


cars

[{'reference': '2022 Jaguar F-PACE S',
  'car_dealer_name': 'Howard Orloff Jaguar, Volvo Cars, Land Rover',
  'price': '$62,535',
  'img_url': None},
 {'reference': '2023 Jaguar F-TYPE R',
  'car_dealer_name': 'Jaguar Land Rover Norwood',
  'price': '$134,575',
  'img_url': None},
 {'reference': '2023 Jaguar F-TYPE R',
  'car_dealer_name': 'Rusnak/Pasadena Jaguar',
  'price': '$132,420',
  'img_url': 'https://platform.cstatic-images.com/large/in/v2/010d674d-9366-50f0-8a29-694ba7779121/59f2b0a9-f51b-41f2-9ada-e7ac341c8c40/-czGMHLhsvsmyZr65jThdrbCBeo.jpg'},
 {'reference': '2023 Jaguar F-TYPE R',
  'car_dealer_name': 'Jaguar Land Rover Dallas',
  'price': '$131,738',
  'img_url': 'https://platform.cstatic-images.com/large/in/v2/4c524a9a-2ffe-53b2-b363-a772e83aef91/107f9948-16a3-4789-9b89-10e9810e4e71/iZG_aj6FzWlaaM2hWXlEHa73jGU.jpg'},
 {'reference': '2023 Jaguar F-TYPE R',
  'car_dealer_name': 'Jaguar Land Rover of Fort Myers',
  'price': '$130,791',
  'img_url': 'https://platform.cstatic

In [20]:
#export data into excel file
import pandas as pd

df = pd.DataFrame(cars)
df


,reference,car_dealer_name,price,img_url
0,2022 Jaguar F-PACE S,"Howard Orloff Jaguar, Volvo Cars, Land Rover","$62,535",None
1,2023 Jaguar F-TYPE R,Jaguar Land Rover Norwood,"$134,575",None
2,2023 Jaguar F-TYPE R,Rusnak/Pasadena Jaguar,"$132,420",https://platform.cstatic-images.com/large/in/v...
3,2023 Jaguar F-TYPE R,Jaguar Land Rover Dallas,"$131,738",https://platform.cstatic-images.com/large/in/v...
4,2023 Jaguar F-TYPE R,Jaguar Land Rover of Fort Myers,"$130,791",https://platform.cstatic-images.com/large/in/v...
...,...,...,...,...
96,2023 Jaguar F-TYPE P450 R-Dynamic,Jaguar of Chattanooga,"$104,825",https://platform.cstatic-images.com/large/in/v...
97,2023 Jaguar F-TYPE P450 R-Dynamic,Jaguar Land Rover South Orlando,"$104,415",https://platform.cstatic-images.com/large/in/v...
98,2023 Jaguar F-TYPE P450 R-Dynamic,Land Rover Jaguar Main Line,"$104,130",https://platform.cstatic-images.com/large/in/v...
99,2023 Jaguar F-TYPE P450 R-Dynamic,Land Rover Jaguar Main Line,"$104,130",https://platform.cstatic-images.com/large/in/v...


In [21]:
df.to_excel(EXPORT_PATH + 'jaguar.xlsx', index = False)